46. 動詞の格フレーム情報の抽出Permalink

45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

作り出す	で は を	会議で ジョンマッカーシーは 用語を

In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

    def __str__(self):
        return f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}"

class Chunk:
    def __init__(self, morphs=None, dst=-1):
        self.morphs = morphs if morphs else []  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst  # 係り先文節インデックス番号
        self.srcs = []  # 係り元文節インデックス番号のリスト

    def __str__(self):
        morphs_surface = ''.join([morph.surface for morph in self.morphs])
        return f"Chunk: {morphs_surface}, dst: {self.dst}, srcs: {self.srcs}"

    def get_text(self):
        return ''.join([morph.surface for morph in self.morphs if morph.pos != '記号'])

    def get_pos(self, pos):
        return [morph for morph in self.morphs if morph.pos == pos]

def parse_cabocha_output(parsed_file):
    sentences = []
    with open(parsed_file, 'r', encoding='utf-8') as f:
        chunks = {}
        chunk = None
        for line in f:
            if line == 'EOS\n':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                if chunks:
                    sorted_chunks = sorted(chunks.items())
                    sentence = [chunk for idx, chunk in sorted_chunks]
                    for idx, chunk in sorted_chunks:
                        if chunk.dst != -1:
                            sentence[chunk.dst].srcs.append(idx)
                    sentences.append(sentence)
                chunks = {}
                chunk = None
            elif line[0] == '*':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                cols = line.split(' ')
                chunk_idx = int(cols[1])
                dst = int(cols[2].rstrip('D'))
                chunk = Chunk(dst=dst)
            else:
                surface, feature = line.split('\t')
                features = feature.split(',')
                if len(features) >= 7:
                    morph = Morph(surface, features[6], features[0], features[1])
                    chunk.morphs.append(morph)
    return sentences

def extract_case_patterns(parsed_file, output_file):
    sentences = parse_cabocha_output(parsed_file)
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            for chunk in sentence:
                verbs = chunk.get_pos('動詞')
                if verbs:
                    predicate = verbs[0].base
                    particles = []
                    phrases = []
                    for src in chunk.srcs:
                        src_chunk = sentence[src]
                        particles += [morph.surface for morph in src_chunk.morphs if morph.pos == '助詞']
                        phrases.append(src_chunk.get_text())
                    if particles:
                        particles = sorted(set(particles))
                        phrases = [phrase for _, phrase in sorted(zip(particles, phrases))]
                        particles = sorted(set(particles))
                        f.write(f"{predicate}\t{' '.join(particles)}\t{' '.join(phrases)}\n")

# 指定解析結果ファイルのパス
parsed_file = 'ai.ja.txt.parsed'
# 出力ファイルのパス
output_file = 'case_patterns_with_phrases.txt'

# 動詞の格パターンと項を抽出してファイルに保存
extract_case_patterns(parsed_file, output_file)
